🎯 Notebook 4 - Predições Finais no Dataset de Teste

In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import StandardScaler

print("PREDIÇÕES FINAIS - BOOTCAMP_TEST.CSV")
print("="*50)

# ===== 1. CARREGAR MODELO TREINADO =====
print("\n1. Carregando modelo e configurações")
print("-"*40)

try:
    # Carregar modelo final
    modelo_data = joblib.load('../models/modelo_final_bootcamp.pkl')
    
    modelo_nome = modelo_data['modelo_tipo']
    modelo = modelo_data['modelo']
    scaler = modelo_data['scaler']
    feature_names = modelo_data['feature_names']
    failure_names = modelo_data['failure_names']
    thresholds = modelo_data['thresholds_otimizados']
    
    print(f"Modelo carregado: {modelo_nome}")
    print(f"F1-Score de treino: {modelo_data['f1_score_final']:.3f}")
    print(f"Total de features: {len(feature_names)}")
    print(f"Thresholds otimizados: {[f'{t:.2f}' for t in thresholds]}")
    
except FileNotFoundError:
    print("Modelo não encontrado. Execute primeiro os notebooks de treino.")
    exit()

PREDIÇÕES FINAIS - BOOTCAMP_TEST.CSV

1. Carregando modelo e configurações
----------------------------------------
Modelo carregado: XGBoost
F1-Score de treino: 0.222
Total de features: 13
Thresholds otimizados: ['0.10', '0.55', '0.30', '0.60', '0.50']


In [2]:
# ===== 2. CARREGAR DADOS DE TESTE =====
print("\n2. Carregando dados de teste")
print("-"*35)

try:
    df_test = pd.read_csv('../data/raw/bootcamp_test.csv')
    print(f"Dados carregados: {df_test.shape}")
    
    # Estrutura das colunas
    print("Colunas encontradas:")
    for i, col in enumerate(df_test.columns, 1):
        print(f"  {i:2d}. {col}")
    
    # Verificar rótulos
    has_failure_cols = any('Falha' in col for col in df_test.columns)
    if has_failure_cols:
        print("Atenção: arquivo de teste contém rótulos (não esperado).")
    else:
        print("Arquivo de teste sem rótulos, conforme esperado.")
        
except FileNotFoundError:
    print("Arquivo bootcamp_test.csv não encontrado em ../data/raw/")
    exit()


2. Carregando dados de teste
-----------------------------------
Dados carregados: (7173, 9)
Colunas encontradas:
   1. id
   2. id_produto
   3. tipo
   4. temperatura_ar
   5. temperatura_processo
   6. umidade_relativa
   7. velocidade_rotacional
   8. torque
   9. desgaste_da_ferramenta
Arquivo de teste sem rótulos, conforme esperado.


In [3]:
# ===== 3. PREPROCESSAMENTO =====
print("\n3. Aplicando preprocessing do treino")
print("-"*48)

def criar_features_engineered(df):
    """Cria as mesmas features engineering usadas no treino."""
    df_copy = df.copy()
    
    # Preencher missing values
    if 'desgaste_da_ferramenta' in df_copy.columns:
        df_copy['desgaste_da_ferramenta'] = df_copy.groupby('tipo')['desgaste_da_ferramenta'].transform(
            lambda x: x.fillna(x.median())
        )
        df_copy['desgaste_da_ferramenta'].fillna(df_copy['desgaste_da_ferramenta'].median(), inplace=True)
    
    # Features engineering
    df_copy['stress_termico'] = df_copy['temperatura_processo'] - df_copy['temperatura_ar']
    df_copy['potencia_mecanica'] = df_copy['torque'] * df_copy['velocidade_rotacional'] / 1000
    df_copy['taxa_desgaste'] = df_copy['desgaste_da_ferramenta'] / (df_copy['velocidade_rotacional'] + 1)
    df_copy['stress_mecanico'] = (df_copy['torque'] * df_copy['desgaste_da_ferramenta']) / (df_copy['velocidade_rotacional'] + 1)
    
    print("Features engineering criadas: stress_termico, potencia_mecanica, taxa_desgaste, stress_mecanico")
    return df_copy

df_test_processed = criar_features_engineered(df_test)

# One-hot encoding
df_test_encoded = pd.get_dummies(df_test_processed, columns=['tipo'], prefix='tipo')

# Ajustar colunas faltantes
for col in feature_names:
    if col not in df_test_encoded.columns:
        df_test_encoded[col] = 0

# Selecionar features
X_test = df_test_encoded[feature_names]

# Normalização
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
print(f"Preprocessing concluído: {X_test_scaled.shape}")


3. Aplicando preprocessing do treino
------------------------------------------------
Features engineering criadas: stress_termico, potencia_mecanica, taxa_desgaste, stress_mecanico
Preprocessing concluído: (7173, 13)


C:\Users\mdc\AppData\Local\Temp\ipykernel_31248\2124250244.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['desgaste_da_ferramenta'].fillna(df_copy['desgaste_da_ferramenta'].median(), inplace=True)


In [4]:
# ===== 4. FAZER PREDIÇÕES =====
print("\n4. Fazendo predições com o modelo final")
print("-"*42)

if modelo_nome == "Random Forest":
    pred_proba_raw = modelo.predict_proba(X_test_scaled)
    pred_proba = np.zeros((len(X_test_scaled), len(failure_names)))
    for i in range(len(failure_names)):
        pred_proba[:, i] = pred_proba_raw[i][:, 1]
else:
    pred_proba_list = [modelo[name].predict_proba(X_test_scaled)[:, 1] for name in failure_names]
    pred_proba = np.array(pred_proba_list).T

# Aplicar thresholds
pred_binarias = np.zeros_like(pred_proba, dtype=int)
for i, threshold in enumerate(thresholds):
    pred_binarias[:, i] = (pred_proba[:, i] >= threshold).astype(int)
    print(f"{failure_names[i]}: {pred_binarias[:, i].sum()} falhas preditas (threshold {threshold:.2f})")

# Estatísticas gerais
total_amostras = len(X_test_scaled)
total_falhas_preditas = pred_binarias.sum()
amostras_com_falha = (pred_binarias.sum(axis=1) > 0).sum()

print(f"\nEstatísticas gerais:")
print(f"  Total de amostras: {total_amostras}")
print(f"  Total de falhas preditas: {total_falhas_preditas}")
print(f"  Amostras com ≥1 falha: {amostras_com_falha}")


4. Fazendo predições com o modelo final
------------------------------------------
FDF: 53 falhas preditas (threshold 0.10)
FDC: 146 falhas preditas (threshold 0.55)
FP: 43 falhas preditas (threshold 0.30)
FTE: 40 falhas preditas (threshold 0.60)
FA: 6 falhas preditas (threshold 0.50)

Estatísticas gerais:
  Total de amostras: 7173
  Total de falhas preditas: 288
  Amostras com ≥1 falha: 267


In [5]:
# ===== 5. DETALHAMENTO POR TIPO =====
print("\n5. Detalhamento por tipo de falha")
print("-"*40)
print(f"{'Tipo':<6} | {'Pred':>4} | {'Taxa':>6} | {'Prob.Média':>10}")
print("-"*40)
for i, name in enumerate(failure_names):
    pred_count = pred_binarias[:, i].sum()
    pred_rate = pred_count / total_amostras * 100
    avg_proba = pred_proba[:, i].mean()
    print(f"{name:<6} | {pred_count:>4} | {pred_rate:>5.1f}% | {avg_proba:>10.3f}")


5. Detalhamento por tipo de falha
----------------------------------------
Tipo   | Pred |   Taxa | Prob.Média
----------------------------------------
FDF    |   53 |   0.7% |      0.004
FDC    |  146 |   2.0% |      0.079
FP     |   43 |   0.6% |      0.006
FTE    |   40 |   0.6% |      0.007
FA     |    6 |   0.1% |      0.005


In [6]:
# ===== 6. CASOS CRÍTICOS =====
print("\n6. Identificação de casos críticos")
df_test['risco_maximo'] = pred_proba.max(axis=1)
casos_criticos = df_test.nlargest(10, 'risco_maximo')
print(casos_criticos[['id', 'id_produto', 'tipo', 'risco_maximo']])

# Distribuição de risco
risco_max = df_test['risco_maximo']
print("\nDistribuição de risco:")
print(f"  Alto (>0.7): {(risco_max > 0.7).sum()}")
print(f"  Médio (0.3-0.7): {((risco_max >= 0.3) & (risco_max <= 0.7)).sum()}")
print(f"  Baixo (<0.3): {(risco_max < 0.3).sum()}")



6. Identificação de casos críticos
         id id_produto tipo  risco_maximo
4523  39783     L51071    L      0.999855
6620  41880     L57008    L      0.999828
6988  42248     M19720    M      0.999661
2116  37376     L50600    L      0.999624
6296  41556     L56849    L      0.999607
6964  42224     M15876    M      0.999557
2394  37654     L47625    L      0.999442
916   36176     L47428    L      0.999367
2509  37769     L54112    L      0.999222
3020  38280     L56736    L      0.999014

Distribuição de risco:
  Alto (>0.7): 181
  Médio (0.3-0.7): 293
  Baixo (<0.3): 6699


In [7]:
# ===== 7. SALVAR RESULTADOS =====
print("\n7. Salvando resultados")
os.makedirs('../results', exist_ok=True)

resultados = df_test[['id', 'id_produto', 'tipo']].copy()
for i, name in enumerate(failure_names):
    resultados[f'pred_{name}'] = pred_binarias[:, i]
    resultados[f'proba_{name}'] = pred_proba[:, i].round(4)
resultados['risco_maximo'] = risco_max.round(4)
resultados['tem_falha_predita'] = (pred_binarias.sum(axis=1) > 0).astype(int)

resultados.to_csv('../results/predicoes_bootcamp_test.csv', index=False)
resultados[resultados['risco_maximo'] > 0.3].to_csv('../results/casos_criticos.csv', index=False)
sumario_api = pd.DataFrame({'id': df_test['id']})
for i, name in enumerate(failure_names):
    sumario_api[name] = pred_binarias[:, i]
sumario_api.to_csv('../results/predicoes_para_api.csv', index=False)

print("Resultados salvos com sucesso.")


7. Salvando resultados
Resultados salvos com sucesso.


In [8]:
# ===== 8. PREVIEW =====
print("\n8. Preview dos resultados")
colunas_mostrar = ['id', 'id_produto', 'tipo'] + [f'pred_{name}' for name in failure_names] + ['risco_maximo']
print(resultados[colunas_mostrar].head())


8. Preview dos resultados
      id id_produto tipo  pred_FDF  pred_FDC  pred_FP  pred_FTE  pred_FA  \
0  35260     L53432    L         0         0        0         0        0   
1  35261     M19544    M         0         0        0         0        0   
2  35262     M16591    M         0         0        0         0        0   
3  35263     L51922    L         0         0        0         0        0   
4  35264     L51701    L         0         0        0         0        0   

   risco_maximo  
0        0.0203  
1        0.0471  
2        0.0357  
3        0.1270  
4        0.0223  


In [9]:
# ===== 9. RESUMO FINAL =====
print("\nResumo final")
resumo_final = f"""
DADOS PROCESSADOS:
  Amostras: {total_amostras}
  Modelo usado: {modelo_nome}
  Features: {len(feature_names)}

PREDIÇÕES:
  Total de falhas preditas: {total_falhas_preditas}
  Casos críticos (risco >30%): {len(resultados[resultados['risco_maximo'] > 0.3])}

ARQUIVOS GERADOS:
  predicoes_bootcamp_test.csv
  casos_criticos.csv
  predicoes_para_api.csv

PRÓXIMOS PASSOS:
  - Enviar predicoes_para_api.csv para API
  - Investigar casos críticos
  - Comparar performance final com outros alunos
"""
print(resumo_final)
print("Predições concluídas com sucesso.")


Resumo final

DADOS PROCESSADOS:
  Amostras: 7173
  Modelo usado: XGBoost
  Features: 13

PREDIÇÕES:
  Total de falhas preditas: 288
  Casos críticos (risco >30%): 473

ARQUIVOS GERADOS:
  predicoes_bootcamp_test.csv
  casos_criticos.csv
  predicoes_para_api.csv

PRÓXIMOS PASSOS:
  - Enviar predicoes_para_api.csv para API
  - Investigar casos críticos
  - Comparar performance final com outros alunos

Predições concluídas com sucesso.
